# EEG - Flow

## 3. Manually select bad channels

The bad channels and the bad segments must be annotated and excluded from subsequent analysis. 

Last edit: 13.04.2023 22:28

In [ ]:
from itertools import chain

from mne import read_annotations
from mne.io import read_raw_fif, write_info, read_info
from mne.preprocessing import compute_bridged_electrodes, interpolate_bridged_electrodes
from pyprep import NoisyChannels

from eeg_flow.config import load_config
from eeg_flow.utils.annotations import merge_bad_annotations
from eeg_flow.utils.bids import get_fname, get_folder
from eeg_flow.utils.concurrency import lock_files
from eeg_flow.viz import plot_bridged_electrodes


_, derivatives_folder, experimenter = load_config()

The parameters of the file to process are defined below. Locks are created to prevent someone else from running the same task and from writing the same derivatives.

In [ ]:
participant = 3  # int
group =  1  # int
task = "oddball"  # str
run = 1  # int

derivatives_folder_preprocessed_p = get_folder(derivatives_folder / "preprocessed", participant, group)
fname_stem = get_fname(participant, group, task, run)

# create locks
derivatives = (
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step3_"+experimenter+"_info.fif"),
    derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step3_"+experimenter+"_oddball_with_bads_annot.fif"),
)
locks = lock_files(*derivatives)

# load raw recording
raw = read_raw_fif(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step1_raw.fif"), preload=True)
annot = read_annotations(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_oddball_with_bads_annot.fif"))
info = read_info(derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step2_info.fif"))

#is this necessary here?
raw.annotations.__add__(annot)

##merge infos? if necessary?

## 2.1 Visual inspection and annotations of bad channels


In [ ]:
raw.filter(
    l_freq=1.0,
    h_freq=40.0,
    picks="eeg",
    method="fir",
    phase="zero-double",
    fir_window="hamming",
    fir_design="firwin",
    pad="edge",
)

In [ ]:
raw.plot(theme="light")

## 2.4 Save derivatives

The updated annotations can now be saved alongside the selected bad channels.

In [ ]:
#what should be saved here?

In [ ]:
fname_info = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step3_"+experimenter+"_info.fif")
assert not fname_info.exists()  # write_info always overwrites 
write_info(fname_info, raw.info)
fname_annot = derivatives_folder_preprocessed_p / fname_stem / (fname_stem + "_step3_"+experimenter+"_oddball_with_bads_annot.fif")
annotations.save(fname_annot, overwrite=False)


Regardless of the success of the task, the locks must be released.
If this step is forgotten, someone might have to remove the corresponding `.lock` file manually.

In [ ]:
for lock in locks:
    lock.release()
del locks  # delete would release anyway